# 💡 Basic Usage Tutorial

Welcome to the Roommate Basic Usage Tutorial! This notebook will guide you through the essential features you need to get started with your intelligent assistant.

## 🎯 Learning Objectives

By the end of this tutorial, you'll know how to:
- ✅ Set up and connect to Roommate
- ✅ Send basic chat messages
- ✅ Understand how Roommate learns about you
- ✅ Use memory-enhanced conversations
- ✅ Perform basic troubleshooting

## 🔧 Step 1: Setup and Connection

First, let's make sure you can connect to your Roommate system.

In [ ]:
# Import required libraries
import requests
import json
import os
from datetime import datetime

# Configuration - Update these if needed
API_BASE_URL = "http://localhost:3000"
API_PASSWORD = os.getenv("ROOMMATE_API_PASSWORD", "your-api-password")

# Set up headers for API requests
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_PASSWORD}"
}

print("🔧 Roommate Basic Setup")
print("=" * 30)
print(f"Server URL: {API_BASE_URL}")
print(f"API Key: {'*' * (len(API_PASSWORD) - 4)}{API_PASSWORD[-4:]}")
print("\n🔍 Testing connection...")

# Test the connection
try:
    response = requests.get(f"{API_BASE_URL}/health", timeout=5)
    if response.status_code == 200:
        print("✅ Connection successful!")
        health_data = response.json()
        print(f"   Server status: {health_data.get('status', 'Running')}")
        print(f"   Version: {health_data.get('version', 'Unknown')}")
    else:
        print(f"⚠️ Server responded with status: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to Roommate server")
    print("💡 Make sure the server is running:")
    print("   cd server && bun run index.ts")
except Exception as e:
    print(f"❌ Connection error: {e}")

## 💬 Step 2: Your First Conversation

Let's start with a simple chat interaction to see how Roommate responds.

In [ ]:
def chat_with_roommate(message, user_id="basic-tutorial-user"):
    """Send a message to Roommate and get a response"""
    payload = {
        "message": message,
        "userId": user_id
    }
    
    try:
        response = requests.post(
            f"{API_BASE_URL}/chat/send",
            headers=headers,
            data=json.dumps(payload),
            timeout=10
        )
        
        if response.status_code == 200:
            result = response.json()
            roommate_response = result.get('response', 'No response received')
            print(f"🤖 Roommate: {roommate_response}")
            return result
        else:
            print(f"❌ Chat failed: {response.status_code}")
            print(f"   Error: {response.text}")
            return None
    except Exception as e:
        print(f"❌ Chat error: {e}")
        return None

# Let's have our first conversation!
print("💬 Starting Your First Conversation")
print("=" * 35)

first_messages = [
    "Hello!",
    "What is your name?",
    "What can you help me with?"
]

for i, message in enumerate(first_messages, 1):
    print(f"\n{i}. 👤 You: {message}")
    chat_with_roommate(message)

## 🧠 Step 3: Teaching Roommate About Yourself

One of Roommate's key features is its ability to learn and remember information about you. Let's teach it some basic facts!

In [ ]:
def teach_roommate(information):
    """Teach Roommate information by sending it as a chat message"""
    print(f"📚 Teaching: {information}")
    response = chat_with_roommate(information)
    
    # The memory saving happens automatically when we chat
    # Let's also explicitly save to memory for demonstration
    save_payload = {
        "userId": "basic-tutorial-user",
        "sentence": information
    }
    
    try:
        memory_response = requests.post(
            f"{API_BASE_URL}/memory/save",
            headers=headers,
            data=json.dumps(save_payload)
        )
        if memory_response.status_code == 200:
            result = memory_response.json()
            memories_saved = result.get('memoriesSaved', 0)
            if memories_saved > 0:
                print(f"   💾 Saved {memories_saved} memory item(s)")
    except Exception as e:
        print(f"   ⚠️ Memory save note: {e}")
    
    return response

print("🧠 Teaching Roommate About You")
print("=" * 30)
print("Let's share some basic information so Roommate can get to know you!\n")

# Basic personal information
personal_facts = [
    "My name is Sam",
    "I live in Seattle",
    "I work as a software developer",
    "I have a cat named Luna",
    "I enjoy reading and hiking"
]

for fact in personal_facts:
    teach_roommate(fact)
    print()  # Add some spacing

## 🔍 Step 4: Testing Memory Recall

Now let's see if Roommate remembers what we taught it!

In [ ]:
print("🔍 Testing Memory Recall")
print("=" * 25)
print("Let's ask Roommate questions to see what it remembers!\n")

memory_questions = [
    "What is my name?",
    "Where do I live?",
    "What do I do for work?",
    "Tell me about my pet",
    "What do I like to do for fun?",
    "Do you know anything about me?"
]

for i, question in enumerate(memory_questions, 1):
    print(f"{i}. 👤 You: {question}")
    response = chat_with_roommate(question)
    
    # Check if the response used memory (if this information is available)
    if response and 'memoryUsed' in response:
        if response['memoryUsed']:
            print("   🧠 Memory was used for this response")
        else:
            print("   💭 Response was generated without specific memory")
    print()

## 🎯 Step 5: Practical Usage Examples

Let's try some practical scenarios that show how Roommate's memory makes conversations more natural and helpful.

In [ ]:
print("🎯 Practical Usage Examples")
print("=" * 27)
print("Here are some realistic scenarios where Roommate's memory helps:\n")

practical_scenarios = [
    {
        "context": "Asking for personalized advice",
        "message": "I'm feeling stressed at work. Any suggestions?"
    },
    {
        "context": "Planning activities",
        "message": "What should I do this weekend?"
    },
    {
        "context": "Pet care reminder",
        "message": "Should I take my cat to the vet?"
    },
    {
        "context": "Location-based suggestions",
        "message": "What's a good restaurant near me?"
    },
    {
        "context": "Career-related question",
        "message": "How can I improve my coding skills?"
    }
]

for i, scenario in enumerate(practical_scenarios, 1):
    print(f"{i}. 📋 Scenario: {scenario['context']}")
    print(f"   👤 You: {scenario['message']}")
    chat_with_roommate(scenario['message'])
    print()

## 📊 Step 6: Understanding Your Data

Let's check what information Roommate has learned about you.

In [ ]:
def check_my_memories():
    """Retrieve and display all memories for the current user"""
    try:
        response = requests.get(
            f"{API_BASE_URL}/memory/get",
            headers=headers,
            params={
                "userId": "basic-tutorial-user",
                "prompt": "Tell me everything you know about me"
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            memories = result.get('memories', [])
            
            print(f"📊 Your Data in Roommate's Memory")
            print("=" * 35)
            print(f"Total memories stored: {len(memories)}\n")
            
            if memories:
                # Group memories by type
                memory_types = {}
                for memory in memories:
                    mem_type = memory.get('type', 'other')
                    if mem_type not in memory_types:
                        memory_types[mem_type] = []
                    memory_types[mem_type].append(memory)
                
                for mem_type, mem_list in memory_types.items():
                    print(f"📁 {mem_type.title()} Information:")
                    for memory in mem_list:
                        key = memory.get('key', 'Unknown')
                        value = memory.get('value', 'Unknown')
                        timestamp = memory.get('timestamp', 'Unknown')
                        print(f"   • {key}: {value} (saved: {timestamp})")
                    print()
            else:
                print("🔍 No specific memories found.")
                print("   This might mean the memory system is still learning")
                print("   or the data is stored in a different format.")
        else:
            print(f"❌ Failed to retrieve memories: {response.status_code}")
    except Exception as e:
        print(f"❌ Error checking memories: {e}")

check_my_memories()

## 🔧 Step 7: Basic Troubleshooting

Let's learn how to check if everything is working correctly.

In [ ]:
def basic_health_check():
    """Perform basic health checks on the Roommate system"""
    print("🔧 Basic System Health Check")
    print("=" * 30)
    
    checks = [
        {"name": "Server Health", "url": "/health"},
        {"name": "Chat API", "url": "/chat/send", "method": "POST", 
         "data": {"message": "health check", "userId": "test"}},
        {"name": "Memory API", "url": "/memory/save", "method": "POST",
         "data": {"userId": "test", "sentence": "health check"}}
    ]
    
    results = []
    
    for check in checks:
        try:
            if check.get("method") == "POST":
                response = requests.post(
                    f"{API_BASE_URL}{check['url']}",
                    headers=headers,
                    data=json.dumps(check['data']),
                    timeout=5
                )
            else:
                response = requests.get(
                    f"{API_BASE_URL}{check['url']}",
                    headers=headers,
                    timeout=5
                )
            
            if response.status_code == 200:
                print(f"✅ {check['name']}: Working")
                results.append(True)
            else:
                print(f"⚠️ {check['name']}: HTTP {response.status_code}")
                results.append(False)
        except Exception as e:
            print(f"❌ {check['name']}: Failed ({e})")
            results.append(False)
    
    print(f"\n📊 Health Check Summary: {sum(results)}/{len(results)} checks passed")
    
    if all(results):
        print("🎉 All systems are working correctly!")
    else:
        print("⚠️ Some issues detected. Check server logs for details.")
        print("💡 Common fixes:")
        print("   • Restart the server: cd server && bun run index.ts")
        print("   • Check if MongoDB is running")
        print("   • Verify your API_PASSWORD is correct")

basic_health_check()

## 🎓 Congratulations!

You've completed the Roommate Basic Usage Tutorial! Here's what you've learned:

### ✅ Skills Acquired
- **Connection Setup**: How to connect to your Roommate server
- **Basic Chat**: Sending messages and receiving responses
- **Memory Training**: Teaching Roommate about yourself
- **Memory Recall**: Using stored information in conversations
- **Practical Usage**: Real-world scenarios and applications
- **Health Monitoring**: Basic troubleshooting and system checks

### 🚀 Next Steps

Now that you understand the basics, you can explore more advanced features:

1. **📖 [Memory System Deep Dive](memory_tutorial.ipynb)** - Learn advanced memory features
2. **🏠 [IoT Integration](iot_tutorial.ipynb)** - Connect smart home devices
3. **📚 [Complete Tutorials](tutorials.ipynb)** - Comprehensive feature overview
4. **📋 [API Reference](api_reference.md)** - Detailed API documentation

### 💡 Tips for Daily Use

- **Be Natural**: Talk to Roommate like you would a human assistant
- **Share Context**: The more Roommate knows about you, the better it can help
- **Ask Questions**: Roommate can remember and help with your interests and preferences
- **Give Feedback**: If responses aren't helpful, try rephrasing or providing more context

### 🆘 Getting Help

If you encounter issues:
- 📖 Check the [Troubleshooting Guide](troubleshooting.md)
- ❓ Read the [FAQ](faq.md)
- 🐛 Report bugs on [GitHub Issues](https://github.com/TheusHen/roommate/issues)

## 🎉 Welcome to the Roommate Community!

You're now ready to use Roommate as your intelligent assistant. The system will continue learning about you and improving its responses over time. Enjoy exploring all the features and capabilities!